In [ ]:
import pandas as pd

from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns

sns.set(context='poster')
%matplotlib inline

### Задача 1:

Рассмотрим как можно провести отбор признаков с помощью обучения линейной регрессии и леса с малым количеством деревьев на примере задачи регрессии. Будем использовать датасет <https://www.kaggle.com/datasets/abrambeyer/openintro-possum> и пытаться предсказать возраст оппосумов на основе различных параметров их тела.

Для упрощения задачи избавимся от категориальных признаков

In [ ]:
data = pd.read_csv('possum.csv').dropna()
X = data[data.columns.drop(['sex', 'Pop', 'age'])]
y = data['age']

Разделим выборку на тренировочную и тестовую часть

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Стандартизируем данные для корректной работы линейной регрессии:

In [ ]:
scaler = <...>
X_train_norm = <...>
X_test_norm = <...>

Обучим модель линейной регрессии с L1-регуляризацией на всех данных, посмотрим на метрику качества

In [ ]:
lin_model = <...>
print('Коэффиценты модели:', <...>)
print('Значение MSE на тренировочной выборке:', MSE(lin_model.predict(X_train_norm), y_train))
print('Значение MSE на тестовой выборке:', MSE(lin_model.predict(X_test_norm), y_test))

Проделаем аналогичную процедуру с лесом c небольшим количеством деревьев:

In [ ]:
forest = <...>
print('Важность признаков:', <...>)
print('Значение MSE на тренировочной выборке:', MSE(forest.predict(X_train), y_train))
print('Значение MSE на тестовой выборке:', MSE(forest.predict(X_test), y_test))

Составим 2 списка наиболее важных признаков - один с признаками, наиболее важными для линейной регрессии, второй с наиболее важными для леса

In [ ]:
lin_imp = []
forest_imp = []
for i, column in enumerate(X.columns):
    if lin_model.coef_[i] > 0.1:
        lin_imp.append(i)
    if forest.feature_importances_[i] > 0.1:
        forest_imp.append(column)

Теперь обучим модели только на важных признаках:

In [ ]:
X_train_norm_lin_imp = X_train_norm[:, lin_imp]
X_test_norm_lin_imp = X_test_norm[:, lin_imp]

X_train_forest_imp = X_train[forest_imp]
X_test_forest_imp = X_test[forest_imp]

In [ ]:
lin_model_2 = Lasso(alpha=0.1).fit(X_train_norm_lin_imp, y_train)
print('Коэффиценты модели:', lin_model_2.coef_)
print('Значение MSE на тренировочной выборке:', MSE(lin_model_2.predict(X_train_norm_lin_imp), y_train))
print('Значение MSE на тестовой выборке:', MSE(lin_model_2.predict(X_test_norm_lin_imp), y_test))

In [ ]:
forest_2 = <...>
print('Важность признаков:', <...>)
print('Значение MSE на тренировочной выборке:', MSE(forest_2.predict(X_train_forest_imp), y_train))
print('Значение MSE на тестовой выборке:', MSE(forest_2.predict(X_test_forest_imp), y_test))

**Вывод:** <...>